This codebook creates multiple alternative prompts to elicit responses to various questionnaires from ChatGPT. Prompts only contain surface level changes in order to test the effect of small changes in prompts to outputs (prompt sensitivity).

## Load Packages

In [1]:
import pandas as pd
import numpy as np
import pickle
import os

## Define Functions

### General Parameter

In [2]:
FOLDER = "../data/surveys/"

COLS_META = ["sex", "age", "currentCountry",  "political_opinion", "race", 
        "religion", "education", "income", "social_class"] 

# adjust to survey (can have different scales)
# more information: Intro about our study, explaining what it is about
ALT1 = "For a scientific study, we are collecting responses to various psychological questionnaires. In the following you will be shown statements from a personality test. Please indicate your level of agreement with each statement. There are no right or wrong answers. Use numbers from 1 to 5, with 1 meaning strongly disagree and 5 meaning strongly agree. Respond with a single number. The statement is: "

# semantics/rephraming, replace as much with synonyms, but kept general sense vague
ALT2 = "For an ongoing research project, please express how much you agree with the statement shown at the end. Respond with an integer between 1 and 5, where 1 indicates a strong disagreement and 5 indicates a strong agreement. Respond with a single digit only. Here's the statement:"

#response values (scale)
ALT3 = "For a scientific study, please indicate your level of agreement with the following statement. Use integer numbers from -2 to 2, with -2 meaning strongly disagree and 2 meaning strongly agree. Respond with a single number. The statement is: "

# also test order of items shown
PROMPT_TEXT = "For a scientific study, please indicate your level of agreement with the following statement. Use integer numbers from 1 to 5, with 1 meaning strongly disagree and 5 meaning strongly agree. Respond with a single number. The statement is: "

### Functions

In [3]:
def extractSurveys(data):
    path = FOLDER + data + ".csv"
    df = pd.read_csv(path)
    print(set(COLS_META) <= set(df.columns)) # check if meta information is in dataset
    
    idx = df.columns.tolist().index("sex") #index of last survey item
    cols_items = df.columns.tolist()[:idx]
    print(cols_items[-1]) # print last survey item
    
    df_items = df.dropna(subset=cols_items, axis = 0)
    df_total = df_items.dropna(subset=COLS_META).reset_index(drop=True)
    df_total.to_csv("../data/processed/" + data + "_cleaned.csv", index=False)
    
    item_path = "../data/items/" + data + "_items.csv"
    if not (os.path.isfile(item_path)):
        items_list = pd.DataFrame(cols_items)
        items_list.to_csv(item_path, index=False)
    else:
        pass

    return df_total, cols_items

## BIG5

Prompting is tested here only for the BFI but this pipeline can be extended to any survey (simply adapt the prompts for the respective survey andd repeat the analyses)

In [4]:
d = "bigfive"
df_bigfive, cols_meta_bigfive = extractSurveys(d)

/tmp/ipykernel_1027789/1109698091.py:3: DtypeWarning: Columns (53,63,66) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


True
bigfive_44


In [5]:
items = pd.read_csv("../data/items/" + d + "_items.csv", sep=";")
# check the respective texts in the original questionnaires
for i, prompt_text in enumerate([ALT1,ALT2,ALT3]):  
    prompts = [prompt_text + "\"" + x + "\"" for x in items.item_text]
    #show prompts
    with open("../data/prompts/ALT" + str(i+1) + "_" + d + ".pkl", "wb") as output:
        pickle.dump(prompts, output)